In [ ]:
import os
import re
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# List all pdf files to analyze their names
def list_file_names(relative_path):
    # Join the path of child folder that contains the PDF files with their file names
    directory = os.path.join(os.path.dirname(os.getcwd()), relative_path)
    files = os.listdir(directory)  # List all files in the directory
    return files

child_folder = "Group project/PDF_data"
pdf_names = list_file_names(child_folder)
print(len(pdf_names))
print(pdf_names)

1195
['1-s2.0-037026939091947A-main.pdf', 'BiberianJPjcondensedl.pdf#page=244.pdf', 'afonichev2003.pdf', '3967_What_Is_Cold_Fusion_and_Why_Should_You_C.pdf', 'BiberianJPjcondensedw.pdf#page=135.pdf', 'BiberianJPjcondensedl.pdf#page=287.pdf', 'BiberianJPjcondensedc.pdf#page=53.pdf', 'BiberianJPjcondensedn.pdf#page=238.pdf', '1157_Calorimetric Experiments Supporting the Transmission Resonance Model for Cold Fusion.pdf', 'BiberianJPjcondensedn.pdf#page=210.pdf', '1159_1-s2.0-0001616078900056-main.pdf', '477_1-s2.0-016890029090740W-main.pdf', '10.1016@0013-46869600169-7.pdf', '3697_978-1-4899-1190-2.pdf', 'BiberianJPjcondensedo.pdf#page=28.pdf', 'Bata1989.pdf', 'BiberianJPjcondensedza.pdf', 'BiberianJPjcondensedr.pdf#page=153.pdf', '3293_Some Thoughts on a Simple Mechanism for the 2H 2H 4He Cold Fusion Reaction.pdf', 'BassRWparmenters.pdf', 'BiberianJPjcondensedz.pdf#page=15.pdf', '3627_Y_Sakamoto_1996_J._Phys.__Condens._Matter_8_3229.pdf', 'ARPAElowenergyn.pdf', 'Arat1999a.pdf', 'Biberian

### Step 1: Get all documents that need to be separated by page in a dictionary and separate them

In [ ]:
same_files_dict = {}

for name in pdf_names:
    if "#" in name:
        document, page_str = name.split("#")[0], name.split("#")[1]
        page = int(re.findall(r'\d+', page_str)[0])  # Extract the page "number"
        # print(page[0])
        # Add this document-page pair to the dict
        if document not in same_files_dict:
            same_files_dict[document] = [page]
        else:
            same_files_dict[document].append(page)


print("# of documents that need to be separate: {}".format(len(same_files_dict)))
print(same_files_dict.keys())
# print(sorted(same_files_dict['BiberianJPjcondensedc.pdf']))

# of documents that need to be separate: 28
dict_keys(['BiberianJPjcondensedl.pdf', 'BiberianJPjcondensedw.pdf', 'BiberianJPjcondensedc.pdf', 'BiberianJPjcondensedn.pdf', 'BiberianJPjcondensedo.pdf', 'BiberianJPjcondensedr.pdf', 'BiberianJPjcondensedz.pdf', 'BiberianJPjcondensedv.pdf', 'BiberianJPjcondensedy.pdf', 'BiberianJPjcondensedp.pdf', 'BiberianJPjcondensede.pdf', 'BiberianJPjcondensedb.pdf', 'BiberianJPjcondensedm.pdf', 'BiberianJPjcondensedg.pdf', 'BiberianJPjcondensedd.pdf', 'BiberianJPjcondensedq.pdf', 'BiberianJPjcondensed.pdf', 'BiberianJPjcondensedt.pdf', 'BiberianJPjcondensedza.pdf', 'BiberianJPjcondensedk.pdf', 'BiberianJPjcondenseds.pdf', 'BiberianJPjcondensedx.pdf', 'BiberianJPjcondensedu.pdf', 'BiberianJPjcondensedh.pdf', 'BiberianJPjcondensedj.pdf', 'BiberianJPjcondensedf.pdf', 'BiberianJPjcondensedi.pdf', 'BiberianJPjcondenseda.pdf'])


Separate all papers in the 28 long documents and save the separated files into TXT files

In [ ]:
import fitz

def separate_papers(document):
    document_path = "PDF_data/" + document
    page_breaks = sorted(same_files_dict[document])
    doc = fitz.open(document_path)

    for i in range(len(page_breaks) - 1):
        start_page = page_breaks[i] - 1
        end_page = page_breaks[i+1] - 1
        text_content = []

        # Extract tect from the defined range
        for page_num in range(start_page, end_page):
            page = doc.load_page(page_num)
            text_content.append(page.get_text())

        # Create a file for each section
        base_filename = document.split(".")[0]  # Precompute the file name to avoid issues in f-string
        output_file_path = f"txt_data/{base_filename}_{page_breaks[i]}.txt"

        # Create a file for each section
        with open(output_file_path, "w", encoding="utf-8") as file:
            file.write("\n".join(text_content))

    doc.close()

In [ ]:
long_docs = list(same_files_dict.keys())

for document in long_docs:
    separate_papers(document)

### Step 2: Rename all files

In [ ]:
import os

def rename_files_sequentially(directory):
    # Initialize counters for each file type
    pdf_counter = 1
    txt_counter = 1

    # List all files in the given directory
    for filename in sorted(os.listdir(directory)):
        # Full path of the current file
        old_file = os.path.join(directory, filename)

        # Check the extension to filter out PDF and TXT files
        if filename.endswith(".pdf"):
            # Form the new filename with the corresponding counter
            new_filename = f"doc{pdf_counter}.pdf"
            pdf_counter += 1
        elif filename.endswith(".txt"):
            # Form the new filename with the corresponding counter
            new_filename = f"doc{txt_counter}.txt"
            txt_counter += 1
        else:
            # Skip files that are neither PDF nor TXT
            continue

        # Full path for the new file
        new_file = os.path.join(directory, new_filename)
        # Rename the file
        os.rename(old_file, new_file)

In [ ]:
# PDF files
pdf_path = "PDF_data/"
rename_files_sequentially(pdf_path)

# txt files
txt_path = "txt_data/"
rename_files_sequentially(txt_path)

### Step 3: Preprocessing

#### Extract titles & abstracts (Not used in the final result)

In [ ]:
import pdfplumber
import pandas as pd
import numpy as np
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', 500)

PDF files

In [ ]:
import os
import re
import pdfplumber

# Create patterns for multiple keywords
abstract_intro_pattern = re.compile(r'\b(Abstract|Introduction)\b', re.IGNORECASE)

def read_pdfs(directory, pre_titles_pdf, error_pdfs):
    files = os.listdir(directory)
    files.sort()

    for filename in files:
        if filename.endswith('.pdf'):
            filepath = os.path.join(directory, filename)
            try:
                with pdfplumber.open(filepath) as pdf:
                    found = False  # Flag to indicate if the title has been found
                    pages_to_read = min(2, len(pdf.pages))  # Read up to two pages, based on the document length

                    for i in range(pages_to_read):
                        page = pdf.pages[i]
                        text = page.extract_text(x_tolerance=2, y_tolerance=2)
                        if text:
                            sentence_list = text.split('\n')
                            for j, line in enumerate(sentence_list):
                                if abstract_intro_pattern.search(line) :
                                    pre_titles_pdf[filename] = sentence_list[:j]
                                    found = True
                                    break
                        if found:
                            break

            except Exception as e:
                print(f"Failed to process {filename}: {e}")
                error_pdfs.append(filename)

In [ ]:
pdf_path = "PDF_data/"
pre_titles_pdf = {}
error_pdfs = []
read_pdfs(pdf_path, pre_titles_pdf, error_pdfs)

Failed to process doc282.pdf: No /Root object! - Is this really a PDF?


In [ ]:
print(len(pre_titles_pdf))
# print(sorted(list(pre_titles_pdf.keys())))
# pre_titles_pdf['doc104.pdf']

# Find out documents that fail to extract pre-titles
succeed_pdfs = []
for k in pre_titles_pdf.keys():
    succeed_pdfs.append(int(k.split(".")[0][3:]))

print(len(succeed_pdfs))
print(sorted(succeed_pdfs))

# for i in range()

461
461
[3, 4, 5, 7, 9, 10, 11, 12, 13, 15, 16, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 59, 60, 61, 63, 64, 71, 73, 74, 76, 77, 79, 81, 82, 83, 85, 87, 88, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 104, 105, 108, 110, 111, 112, 113, 114, 117, 119, 120, 121, 122, 123, 124, 125, 127, 128, 129, 130, 132, 133, 135, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 164, 165, 166, 167, 170, 171, 173, 174, 175, 176, 177, 178, 179, 181, 182, 185, 186, 188, 189, 190, 191, 192, 197, 198, 199, 200, 202, 203, 205, 207, 211, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 225, 226, 227, 228, 231, 234, 237, 243, 245, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 260, 261, 262, 263, 266, 269, 270, 271, 272, 273, 274, 277, 279, 285, 286, 287, 289, 291, 293, 296, 298, 299, 300, 303, 304, 306, 308, 309, 310, 312, 315,

#### Extract Content

PDF files

In [ ]:
intro_pattern = re.compile(r'introduction', re.IGNORECASE)

def extract_content_from_pdf(filepath):
    content_started = False
    all_content = []
    all_content_by_page = []
    try:
        with pdfplumber.open(filepath) as pdf:
            for page in pdf.pages:
                text = page.extract_text(x_tolerance=2, y_tolerance=2)
                if text:
                    if not content_started:
                        lines = text.split('\n')
                        for i, line in enumerate(lines):
                            if intro_pattern.search(line):
                                all_content.extend(lines[i+1:])  # Add rest of the page after 'Introduction'
                                all_content_by_page.append('\n'.join(lines[i+1:]))  # Start the page-wise content
                                content_started = True
                                break
                    else:
                        all_content.append(text)  # Add entire page text
                        all_content_by_page.append(text)  # Add entire page text to "page-wise" content
        return filepath, '\n'.join(all_content), all_content_by_page
    except Exception as e:
        return filepath, "Error: " + str(e), []

# Use python threads to run mulitple PDF files at the same time and join all the results to content_dict and content_dict_by_page at last
def read_contents(directory):
    content_dict = {}
    content_dict_by_page = {}
    error_pdfs = []

    files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.pdf')]
    files.sort()

    with ThreadPoolExecutor() as executor:
        results = executor.map(extract_content_from_pdf, files)
        for result in results:
            filepath, content, pages_content = result
            filename = os.path.basename(filepath)
            if isinstance(content, str):
                content_dict[filename] = content
                content_dict_by_page[filename] = pages_content
            else:
                error_pdfs.append(filename)

    return content_dict, content_dict_by_page, error_pdfs

In [ ]:
pdf_path = "PDF_data/"
content_dict, content_dict_by_page, error_pdfs = read_contents(pdf_path)

In [ ]:
print(len(content_dict))
print(len(content_dict_by_page))

1193
1193


In [ ]:
content_df = pd.DataFrame(list(content_dict.items()), columns=['Filename', 'Content'])
content_by_page_df = pd.DataFrame(list(content_dict_by_page.items()), columns=['Filename', 'Content'])

# Save all extracted documents to csv files
content_df.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/pdf_content_0504.csv")
content_by_page_df.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/pdf_content_by_page_0504.csv")

#### Read the extracted and saved documents from csv

In [ ]:
content_by_page_df = pd.read_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/pdf_content_by_page_0504.csv")
content_df = pd.read_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/pdf_content_0504.csv")

print(content_by_page_df.shape)
print(content_df.shape)

(1193, 3)
(1193, 3)


In [ ]:
content_by_page_dict = content_by_page_df.set_index('Filename')['Content'].to_dict()
content_dict = content_df.set_index('Filename')['Content'].to_dict()

print(content_by_page_dict.keys())
docs = [int(k.split(".")[0][3:]) for k in content_by_page_dict.keys()]
print(sorted(docs))

dict_keys(['doc1.pdf', 'doc10.pdf', 'doc100.pdf', 'doc1000.pdf', 'doc1001.pdf', 'doc1002.pdf', 'doc1003.pdf', 'doc1004.pdf', 'doc1005.pdf', 'doc1006.pdf', 'doc1007.pdf', 'doc1008.pdf', 'doc1009.pdf', 'doc101.pdf', 'doc1010.pdf', 'doc1011.pdf', 'doc1012.pdf', 'doc1013.pdf', 'doc1014.pdf', 'doc1015.pdf', 'doc1016.pdf', 'doc1017.pdf', 'doc1018.pdf', 'doc1019.pdf', 'doc102.pdf', 'doc1020.pdf', 'doc1021.pdf', 'doc1022.pdf', 'doc1023.pdf', 'doc1024.pdf', 'doc1025.pdf', 'doc1026.pdf', 'doc1027.pdf', 'doc1028.pdf', 'doc1029.pdf', 'doc103.pdf', 'doc1030.pdf', 'doc1031.pdf', 'doc1032.pdf', 'doc1033.pdf', 'doc1034.pdf', 'doc1035.pdf', 'doc1036.pdf', 'doc1037.pdf', 'doc1038.pdf', 'doc1039.pdf', 'doc104.pdf', 'doc1040.pdf', 'doc1041.pdf', 'doc1042.pdf', 'doc1043.pdf', 'doc1044.pdf', 'doc1045.pdf', 'doc1046.pdf', 'doc1047.pdf', 'doc1048.pdf', 'doc1049.pdf', 'doc105.pdf', 'doc1050.pdf', 'doc1051.pdf', 'doc1052.pdf', 'doc1053.pdf', 'doc1054.pdf', 'doc1055.pdf', 'doc1056.pdf', 'doc1057.pdf', 'doc1058.p

In [ ]:
# Find out the documents that can't be parsed successfully
error_docs = []

for k, v in content_by_page_dict.items():
    if v != '[]':
        error_docs.append(int(k.split(".")[0][3:]))

print("# of documents failed to be parsed successfully: {}".format(len(error_docs)))
print(sorted(error_docs))

# of documents failed to be parsed successfully: 827
[3, 4, 7, 9, 10, 11, 12, 13, 17, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 48, 49, 50, 52, 53, 54, 56, 57, 58, 59, 60, 61, 63, 64, 67, 68, 69, 71, 72, 73, 76, 77, 80, 81, 82, 87, 88, 91, 92, 93, 94, 95, 98, 99, 101, 102, 103, 104, 108, 110, 111, 112, 113, 114, 117, 119, 120, 121, 122, 123, 124, 128, 129, 130, 132, 133, 135, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 160, 161, 162, 164, 165, 166, 167, 169, 170, 171, 173, 175, 177, 178, 181, 182, 184, 185, 186, 187, 188, 189, 190, 191, 192, 197, 198, 199, 200, 202, 203, 204, 205, 207, 211, 214, 215, 216, 217, 218, 219, 225, 226, 227, 228, 234, 243, 245, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 260, 262, 263, 266, 269, 272, 273, 274, 277, 279, 280, 286, 287, 289, 292, 293, 294, 296, 297, 298, 299, 300, 302, 306, 307, 309, 310, 312, 313, 315, 319, 320, 322, 323, 32

#### 827 documents are parsed successfully

In [ ]:
content_by_page_df.head(10)
# content_by_page_df.dtypes

# Create a mask where 'Content' column is not an empty list using apply
mask = content_by_page_df['Content'].apply(lambda x: x != '[]')

# Apply the mask to filter the DataFrame
filtered_df = content_by_page_df[mask]

# Display the filtered DataFrame
print(filtered_df.shape)
filtered_df.head(10)

(827, 3)


,Unnamed: 0,Filename,Content
1,1,doc10.pdf,['Before presenting new results we shall describe some general features\nof the hydrogen high pressure technique as applied to investigations of\nmetal-hydrogen systems. This technique enables metallic systems to be\nreacted with a thermodynamically well-defined highly active reference\nphase (gaseous hydrogen). Two factors are particularly important.\n(1) The reaction between gaseous hydrogen and metallic surfaces can\nbe allowed to proceed for any length of time. This means that even proce...
3,3,doc1000.pdf,['Palladium hydrogen system has been intensively investigated due to the exceptional ability of palladium to adsorb and\nabsorb hydrogen. Achieving consistent high loading of Hydrogen (H) and Deuterium (D) in Palladium (Pd) is one\nof the most challenging issues in the field of Anomalous Heat Effect (AHE). Numerous researchers have suggested\nthat the concentration of deuterium loaded into palladium may have to exceed a critical value before the generation of\nexcess heat [1–3]. Studies show...
4,4,doc1001.pdf,['Palladium hydrogen system has been intensively investigated due to the exceptional ability of palladium to adsorb and\nabsorb hydrogen. Achieving consistent high loading of Hydrogen (H) and Deuterium (D) in Palladium (Pd) is one\nof the most challenging issues in the field of Anomalous Heat Effect (AHE). Numerous researchers have suggested\nthat the concentration of deuterium loaded into palladium may have to exceed a critical value before the generation of\nexcess heat [1–3]. Studies show...
5,5,doc1002.pdf,['Palladium hydrogen system has been intensively investigated due to the exceptional ability of palladium to adsorb and\nabsorb hydrogen. Achieving consistent high loading of Hydrogen (H) and Deuterium (D) in Palladium (Pd) is one\nof the most challenging issues in the field of Anomalous Heat Effect (AHE). Numerous researchers have suggested\nthat the concentration of deuterium loaded into palladium may have to exceed a critical value before the generation of\nexcess heat [1–3]. Studies show...
6,6,doc1003.pdf,"['The March 23, 1989 announcement by Martin Fleischmann and Stanley Pons of the possibility of nuclear fusion in an\nelectrochemical system of heavy water (D O) and Pd generated worldwide interest [1–3]. Since then, small groups\n2\nof researchers have been trying to replicate these claims without definitive success. Some experimental observations\nof anomalous heat effects (AHE) were made using calibrated calorimetry [4]. Others have claimed the correlated\ngeneration of He as a reaction by..."
7,7,doc1004.pdf,"['The March 23, 1989 announcement by Martin Fleischmann and Stanley Pons of the possibility of nuclear fusion in an\nelectrochemical system of heavy water (D O) and Pd generated worldwide interest [1–3]. Since then, small groups\n2\nof researchers have been trying to replicate these claims without definitive success. Some experimental observations\nof anomalous heat effects (AHE) were made using calibrated calorimetry [4]. Others have claimed the correlated\ngeneration of He as a reaction by..."
8,8,doc1005.pdf,"['The March 23, 1989 announcement by Martin Fleischmann and Stanley Pons of the possibility of nuclear fusion in an\nelectrochemical system of heavy water (D O) and Pd generated worldwide interest [1–3]. Since then, small groups\n2\nof researchers have been trying to replicate these claims without definitive success. Some experimental observations\nof anomalous heat effects (AHE) were made using calibrated calorimetry [4]. Others have claimed the correlated\ngeneration of He as a reaction by..."
9,9,doc1006.pdf,"['The March 23, 1989 announcement by Martin Fleischmann and Stanley Pons of the possibility of nuclear fusion in an\nelectrochemical system of heavy water (D O) and Pd generated worldwide interest [1–3]. Since then, small groups\n2\nof researchers have been trying to replicate these claims without definitive success. Some experimental o

In [ ]:
# Convert the string back to a list to separate pages
import ast

print(len(filtered_df))
filtered_dict = filtered_df.set_index('Filename')['Content'].to_dict()
content_by_page_new = {}

for k, v in filtered_dict.items():
    v = str(v).replace("\\n", " ")
    content_by_page_new[k] = ast.literal_eval(v)

print(len(content_by_page_new))

827
827


In [ ]:
# Choose the most suitable 400 documents to train the model (with fewer word count on each page)
document_stats = {}

for doc_name, pages in content_by_page_new.items():
    num_pages = len(pages)
    total_words = 0
    words_per_page = []

    for page in pages:
        # Splitting each page into words.
        word_count = len(page.split(" "))
        total_words += word_count

    document_stats[doc_name] = {
        'number_of_pages': num_pages,
        'avg_words_per_page': round(total_words/num_pages),
        'total_words': total_words
    }

document_stats_df = pd.DataFrame(document_stats)
document_stats_df = document_stats_df.T
document_stats_df.sort_values(by='number_of_pages').head(10)

# document_stats_df.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/document_stats.csv")  # Store the document stats to csv

,number_of_pages,avg_words_per_page,total_words
doc626.pdf,1,1,1
doc228.pdf,1,725,725
doc1159.pdf,1,627,627
doc302.pdf,1,70,70
doc307.pdf,1,225,225
doc698.pdf,1,11,11
doc333.pdf,2,140,281
doc294.pdf,2,143,286
doc226.pdf,2,662,1323
doc451.pdf,2,480,961


Select about 400 most suitable documents to train:  
Criteria: at least 5 pages + fewer avg_words_per_page

In [ ]:
selected_docs = document_stats_df[document_stats_df['number_of_pages'] >= 5]
print("There are {} documents with more than 5 pages".format(selected_docs.shape[0]))

# Sort by avg_words_per_page: Select documents with word count between 280 and 400 (For better document quality)
selected_docs = selected_docs[(selected_docs["avg_words_per_page"] <= 1000) & (selected_docs["avg_words_per_page"] >= 100)]
selected_docs = selected_docs[selected_docs["number_of_pages"] <= 85]
selected_docs_lst = list(selected_docs.index)
print("# of selected document: {}".format(selected_docs.shape[0]))
selected_docs.head(30)

# selected_docs.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/selected_document_stats.csv")  # Store the stats to a csv file

There are 742 documents with more than 5 pages
# of selected document: 366


,number_of_pages,avg_words_per_page,total_words
doc10.pdf,15,318,4766
doc1019.pdf,73,386,28154
doc102.pdf,7,483,3384
doc1020.pdf,73,386,28154
doc1021.pdf,73,386,28154
doc1022.pdf,73,386,28154
doc1023.pdf,73,386,28154
doc1024.pdf,73,386,28154
doc1025.pdf,73,386,28154
doc104.pdf,9,610,5491


In [ ]:
training_data_dict = {}

# First extract all the selected docuaments from content_by_page_new
selected_content_by_page = {}
selected_content_by_page = {k: v for k, v in content_by_page_new.items() if k in selected_docs_lst}

# Add the selected documents and convert them to DataFrame
doc_id = []
content = []
for doc, pages in selected_content_by_page.items():
    for page in pages:
        doc_id.append(doc)
        content.append(page)


print(len(selected_content_by_page))
print(len(doc_id), len(content))

366
6219 6219


In [ ]:
# Create the final dataframe for llama!!
training_data_df = pd.DataFrame({
    'document': doc_id,
    'content': content
})

print(training_data_df.shape)
training_data_df.head()

# Save the training data!
# training_data.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/training_data.csv")

(6219, 2)


,document,content
0,doc10.pdf,Before presenting new results we shall describe some general features of the hydrogen high pressure technique as applied to investigations of metal-hydrogen systems. This technique enables metallic systems to be reacted with a thermodynamically well-defined highly active reference phase (gaseous hydrogen). Two factors are particularly important. (1) The reaction between gaseous hydrogen and metallic surfaces can be allowed to proceed for any length of time. This means that even processes wit...
1,doc10.pdf,"116 Fig. 1. Logarithm of the hydrogen fugacity as a function of the hydrogen pressure at 298 K. behaviour, the same thermodynamic activity of hydrogen can be obtained at pressures five orders of magnitude lower. This comparison illustrates the power of the high pressure technique to achieve high thermodynamic activ- ities of hydrogen. Three different approaches which demonstrate the advantages of high pressure techniques in investigations of metal-hydrogen systems are outlined below. Recent ..."
2,doc10.pdf,"117 this paper; the results obtained by the Soviet group are presented elsewhere. A number of review articles are available in the literature [8 - 131. 2. Experimental methods The general characteristics of the experimental techniques adopted are summarized briefly below. All experiments am carried out under stable hydrostatic conditions with respect to hydrogen. The working volume of the high pressure gas is of the order of 10 cm3. The sample to be investigated, which is in the form of foil..."
3,doc10.pdf,118 0.0s 0.07 2 4 6 8 10 * pkbar) Fig. 2. The relative electrical resistance of an amorphous Ni0.8P0.2 foil as a function of the hydrogen pressure at 298 K. Fig. 3. The electrical resistance of a nickel foil as a function of the hydrogen pressure at 298 K. due to hydride formation just above 6 kbar and a further continuous de- crease characteristic of the hydride phase at higher pressures. Confirmation of the assumption that the changes observed were caused by hydrogen was obtained by perfor...
4,doc10.pdf,119 0.9600 0.9500 I’ j ” 1’ ” ” a L 8 12 16 20 2L * ptkbw) Fig. 4. Relative electrical resistance of amorphous NiP ((3 = -(l/P)(AR/Re) = (1.8 + 0.2) x 1O-7 bar-‘) and crystalline nickel (0 = (2.0 + 0.1) x lop6 bar-‘) foils as a function of the hydrostatic pressure at 298 K (an inert pressure transmitting medium was used). Fig. 5. Relative electrical resistance of an amorphous Nie.64Zre.s6 foil as a function of hydrogen pressure during cycling at 298 K. When the hydrogen pressure is decreased...


### Step 4: Generate prompts for Fine Tuning

Data proprocessing - generate text data to generate Q&A pairs

In [ ]:
# Preprocess documents to comply with the "token number limit" of the T5 tokenizer
fine_tune_data_pre = training_data_df.groupby('document')['content'].agg('\n'.join).reset_index()
print(fine_tune_data_pre.shape)
fine_tune_data_pre.head()

fine_tune_data_dict = fine_tune_data_pre.set_index('document')['content'].to_dict()
print((fine_tune_data_dict.keys()))

(366, 2)
dict_keys(['doc10.pdf', 'doc1019.pdf', 'doc102.pdf', 'doc1020.pdf', 'doc1021.pdf', 'doc1022.pdf', 'doc1023.pdf', 'doc1024.pdf', 'doc1025.pdf', 'doc104.pdf', 'doc1096.pdf', 'doc1097.pdf', 'doc1098.pdf', 'doc1099.pdf', 'doc11.pdf', 'doc1100.pdf', 'doc1101.pdf', 'doc1102.pdf', 'doc1105.pdf', 'doc1110.pdf', 'doc1111.pdf', 'doc1113.pdf', 'doc1114.pdf', 'doc1115.pdf', 'doc1116.pdf', 'doc1117.pdf', 'doc1118.pdf', 'doc1119.pdf', 'doc112.pdf', 'doc1120.pdf', 'doc1125.pdf', 'doc1128.pdf', 'doc113.pdf', 'doc1130.pdf', 'doc1131.pdf', 'doc1132.pdf', 'doc1133.pdf', 'doc1134.pdf', 'doc1135.pdf', 'doc1136.pdf', 'doc1137.pdf', 'doc1138.pdf', 'doc1139.pdf', 'doc114.pdf', 'doc1140.pdf', 'doc1141.pdf', 'doc1142.pdf', 'doc1143.pdf', 'doc1144.pdf', 'doc1146.pdf', 'doc1150.pdf', 'doc1152.pdf', 'doc1153.pdf', 'doc1158.pdf', 'doc1160.pdf', 'doc1161.pdf', 'doc1162.pdf', 'doc1164.pdf', 'doc117.pdf', 'doc1174.pdf', 'doc1176.pdf', 'doc1177.pdf', 'doc1181.pdf', 'doc1184.pdf', 'doc1185.pdf', 'doc1186.pdf', 

In [ ]:
# First select documents that contains the word "LENR"/"cold fusion" to try
LENR_docs = []
for key, content in fine_tune_data_dict.items():
    if "lenr" in content.lower() or "cold fusion" in content.lower():
        LENR_docs.append(key)
print("# of documents with words 'LENR/cold fusion': {}".format(len(LENR_docs)))

# Then select the top 50 documents that contains the word "LENR"/"cold fusion" with "fewer" total word count to create sample data
print("Select the top 50 documents that contain the fewest total word counts with keywords 'LENR/cold fusion'")
selected_doc_stats = document_stats_df.loc[LENR_docs]
selected_LENR_docs_lst = list(selected_doc_stats.sort_values('total_words').head(50).index)
# print(len(selected_LENR_docs_lst))
fine_tune_data_dict = {k: v for k, v in fine_tune_data_dict.items() if k in selected_LENR_docs_lst}
print("Selected documents: ", list(fine_tune_data_dict.keys()))
print("Selected documents num: ", len(list(fine_tune_data_dict.keys())))

# of documents with words 'LENR/cold fusion': 249
Select the top 50 documents that contain the fewest total word counts with keywords 'LENR/cold fusion'
Selected documents:  ['doc1116.pdf', 'doc1119.pdf', 'doc1125.pdf', 'doc1134.pdf', 'doc1136.pdf', 'doc1140.pdf', 'doc1142.pdf', 'doc1176.pdf', 'doc119.pdf', 'doc120.pdf', 'doc123.pdf', 'doc124.pdf', 'doc166.pdf', 'doc19.pdf', 'doc198.pdf', 'doc203.pdf', 'doc211.pdf', 'doc214.pdf', 'doc215.pdf', 'doc216.pdf', 'doc252.pdf', 'doc272.pdf', 'doc286.pdf', 'doc289.pdf', 'doc29.pdf', 'doc31.pdf', 'doc32.pdf', 'doc335.pdf', 'doc349.pdf', 'doc35.pdf', 'doc357.pdf', 'doc379.pdf', 'doc4.pdf', 'doc412.pdf', 'doc454.pdf', 'doc509.pdf', 'doc517.pdf', 'doc528.pdf', 'doc541.pdf', 'doc549.pdf', 'doc557.pdf', 'doc589.pdf', 'doc655.pdf', 'doc688.pdf', 'doc694.pdf', 'doc699.pdf', 'doc704.pdf', 'doc705.pdf', 'doc707.pdf', 'doc88.pdf']
Selected documents num:  50


Separate the paragraphs of each document into "sentences" to comply with the word limit of T5 tokenizer (512 words)

In [ ]:
import nltk
nltk.download('punkt')
from transformers import T5ForConditionalGeneration, T5TokenizerFast

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tingshow_liu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/tingshow_liu/Downloads/NYU BigData/Group project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
def process_paragraphs(doc_dict):
    # Initialize the T5 tokenizer
    tokenizer = T5TokenizerFast.from_pretrained("t5-base")

    # Dictionary to store processed paragraphs
    processed_docs = []
    # Dictionary to store sentences that exceed 512 tokens
    oversized_sentences = []

    # Process each document
    for content in doc_dict.values():
        sentences = nltk.sent_tokenize(content)
        current_paragraph = ""
        current_tokens = []

        for sentence in sentences:
            sentence_tokens = tokenizer.tokenize(sentence)
            sentence_token_count = len(sentence_tokens)

            if sentence_token_count > 512:
                # Directly add to oversized if a single sentence exceeds the limit
                if current_paragraph:
                    processed_docs.append(current_paragraph)
                    current_paragraph = ""
                    current_tokens = []
                oversized_sentences.append(sentence)
                continue

            # If the sentence does not exceed the size limit (512)
            if len(current_tokens) + sentence_token_count <= 512:
                # Concatenate current sentence to the paragraph if within the limit
                current_paragraph += (' ' + sentence if current_paragraph else sentence)
                current_tokens.extend(sentence_tokens)
            else:
                # Add current paragraph to list and start a new one if the concated sentence token count exceeds 512
                processed_docs.append(current_paragraph)
                current_paragraph = sentence
                current_tokens = sentence_tokens

    return processed_docs, oversized_sentences

In [ ]:
processed_docs, oversized_docs = process_paragraphs(fine_tune_data_dict)
# Text data used to generate Q&A pairs with "ThomasSimonini/t5-end2end-question-generation model
print(len(processed_docs))
print(len(oversized_docs))
processed_docs[:5]

350
0


['through a valve and a pressure regulator. The mechanical ar- It has been suggested recently1\'2 that deuterium in hydride- rangement was tested for vacuum and high pressure at both forming metals (i.e., palladium and titanium) can undergo room temperature and liquid nitrogen temperature. nuclear reaction at room temperature. Such a fascinating sup- The palladium sample was attached to a proper copper position has stimulated a variety of experimental and theo- support to ensure good thermal contact during a temperature retical investigations in the scientific community. But, so far, change in the system. The copper support was surrounded by the experimental information is incomplete, and a clear pic- a special dewar during the cooling phase up to the liquid ni- ture of the phenomenon does not yet exist. Furthermore, if trogen temperature and by an electric heater during the heat- we consider the matter from the experimental point of view, ing phase up to — 300°C. An auxiliary line all

Generate Questions with text data in "processed_docs"

In [ ]:
from transformers import T5ForConditionalGeneration, T5TokenizerFast

In [ ]:
hfmodel = T5ForConditionalGeneration.from_pretrained("ThomasSimonini/t5-end2end-question-generation")
tokenizer = T5TokenizerFast.from_pretrained("t5-base")  # t5-small

# Function to generate questions from the given content
def run_model(input_string, **generator_args):
    generator_args = {
    "max_length": 256,
    "num_beams": 4,
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    input_string = "≈: " + input_string + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = hfmodel.generate(input_ids, **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    # output = [item.split("<sep>") for item in output]
    return output

In [ ]:
# Generate questions
questions_lst = []

for text in processed_docs:
    questions = run_model(text)
    questions_lst.extend(questions)

print(len(questions_lst))
questions_lst[:5]

In [ ]:
# Cut the questions
final_questions = []
final_contents = []  # The according content from which the question in final_questions is generated

for i in range(len(questions_lst)):
    questions = questions_lst[i].split("?")
    for q in questions:
        if q != "":
            final_questions.append((q + "?").strip())
            final_contents.append(processed_docs[i])

print(len(final_questions))
print(len(final_contents))
final_questions[:10]

944
944


['How was the palladium sample kept at 77 K under D pres- 2 conditions?',
 'How long was the d-d fusion process rate expected in hydrogenated metals sitions?',
 'When was the system left at room temperature untf+ /-3He + n, (1) der vacuum for 9 h?',
 'What was measured by a liquid d+d-+4 He + 7- (3) NE-213 scintillator?',
 'How were the gamma rays with energy higher than 10 MeV monitored?',
 'What was the overall detection efficiency of the BaF scintillator 113 r V 2 CO 100 /  crystal?',
 'How was the detector placed in front of the palladium sheet?',
 'What was covered by an aluminum absorber to stop protons of —1 MeV of kinetic energy?',
 'What did the connection of the detection apparatus with a personal computer acquisition system allow us to store?',
 'What is the silicon spec- 2 7 days is reported in Fig. 4?']

In [ ]:
final_contents[:10]

['through a valve and a pressure regulator. The mechanical ar- It has been suggested recently1\'2 that deuterium in hydride- rangement was tested for vacuum and high pressure at both forming metals (i.e., palladium and titanium) can undergo room temperature and liquid nitrogen temperature. nuclear reaction at room temperature. Such a fascinating sup- The palladium sample was attached to a proper copper position has stimulated a variety of experimental and theo- support to ensure good thermal contact during a temperature retical investigations in the scientific community. But, so far, change in the system. The copper support was surrounded by the experimental information is incomplete, and a clear pic- a special dewar during the cooling phase up to the liquid ni- ture of the phenomenon does not yet exist. Furthermore, if trogen temperature and by an electric heater during the heat- we consider the matter from the experimental point of view, ing phase up to — 300°C. An auxiliary line all

Generate Answers to the questions generated above with text data in "processed_docs"

In [ ]:
from transformers import pipeline

In [ ]:
# download model (load a smaller, faster model)
qa_pipeline = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')

def generate_answers(question, content):
    # Use the pipeline to answer the question
    answer = qa_pipeline({
        'question': question,
        'context': content
    })

    return answer

In [ ]:
# Generate the answers
answers = []

for i in range(len(final_questions)):
    answers.append(generate_answers(final_questions[i], final_contents[i]))

print(len(answers))
answers[:5]  # List of dictionaries

944


[{'score': 0.07312247157096863,
  'start': 1620,
  'end': 1667,
  'answer': 'by changing temperature and pressure of a palla'},
 {'score': 0.688685953617096, 'start': 1886, 'end': 1890, 'answer': '22 h'},
 {'score': 0.12462595850229263, 'start': 1003, 'end': 1007, 'answer': '1990'},
 {'score': 0.9662830233573914,
  'start': 1084,
  'end': 1093,
  'answer': 'PALLADIUM'},
 {'score': 0.6943314671516418,
  'start': 57,
  'end': 68,
  'answer': '<D / Vacuum'}]

In [ ]:
final_answers = [dictionary['answer'] for dictionary in answers]
print(len(final_answers))
final_answers[:5]

944


['by changing temperature and pressure of a palla',
 '22 h',
 '1990',
 'PALLADIUM',
 '<D / Vacuum']

In [ ]:
final_questions[:5]

['How was the palladium sample kept at 77 K under D pres- 2 conditions?',
 'How long was the d-d fusion process rate expected in hydrogenated metals sitions?',
 'When was the system left at room temperature untf+ /-3He + n, (1) der vacuum for 9 h?',
 'What was measured by a liquid d+d-+4 He + 7- (3) NE-213 scintillator?',
 'How were the gamma rays with energy higher than 10 MeV monitored?']

In [ ]:
# Concat the questions, answers, and contents pairs into a DataFrame
fine_tune_data = {
    "Question": final_questions,
    "Answer": final_answers,
    "Content": final_contents
}

# Create the DataFrame
fine_tune_data_df = pd.DataFrame(fine_tune_data)
print(fine_tune_data_df.shape)
fine_tune_data_df.head()

# Store them!
# fine_tune_data_df.to_csv("/Users/tingshow_liu/Downloads/NYU BigData/Group project/fine_tuning_data.csv")

(944, 3)


,Question,Answer,Content
0,How was the palladium sample kept at 77 K under D pres- 2 conditions?,by changing temperature and pressure of a palla,"through a valve and a pressure regulator. The mechanical ar- It has been suggested recently1'2 that deuterium in hydride- rangement was tested for vacuum and high pressure at both forming metals (i.e., palladium and titanium) can undergo room temperature and liquid nitrogen temperature. nuclear reaction at room temperature. Such a fascinating sup- The palladium sample was attached to a proper copper position has stimulated a variety of experimental and theo- support to ensure good thermal co..."
1,How long was the d-d fusion process rate expected in hydrogenated metals sitions?,22 h,"through a valve and a pressure regulator. The mechanical ar- It has been suggested recently1'2 that deuterium in hydride- rangement was tested for vacuum and high pressure at both forming metals (i.e., palladium and titanium) can undergo room temperature and liquid nitrogen temperature. nuclear reaction at room temperature. Such a fascinating sup- The palladium sample was attached to a proper copper position has stimulated a variety of experimental and theo- support to ensure good thermal co..."
2,"When was the system left at room temperature untf+ /-3He + n, (1) der vacuum for 9 h?",1990,"During this period the during gas absorption in nonequilibrium states, as suggested Pd-D system was expected to change the stoichiometry from by Cassandro et al.5 Pd-Do.3 to Pd-Do.7 and then to pure palladium. The compo- Assuming that d-d nuclear fusion could occur during sitions were tested on various palladium targets prepared in phase transitions, we focused on studying the nuclear prod- the same way and measured by Nuclear Reaction Analysis.9 ucts of the following reactions: 3. Finally, ..."
3,What was measured by a liquid d+d-+4 He + 7- (3) NE-213 scintillator?,PALLADIUM,"During this period the during gas absorption in nonequilibrium states, as suggested Pd-D system was expected to change the stoichiometry from by Cassandro et al.5 Pd-Do.3 to Pd-Do.7 and then to pure palladium. The compo- Assuming that d-d nuclear fusion could occur during sitions were tested on various palladium targets prepared in phase transitions, we focused on studying the nuclear prod- the same way and measured by Nuclear Reaction Analysis.9 ucts of the following reactions: 3. Finally, ..."
4,How were the gamma rays with energy higher than 10 MeV monitored?,<D / Vacuum,"200 / The gamma rays with energy higher than 10 MeV were <D / Vacuum ——11 1 monitored by a 5-cm-diam x 10-cm-long BaF scintillator 113 r V 2 CO 100 / ^ crystal placed quite close and external to the cell. For this de- A k_ tector, we estimated an overall detection efficiency (includ- CD Palladium Q. r ing the geometric efficiency) of ~9 x 10~4 in the energy E CD gamma-ray range between 10 and 30 MeV. Due to this effi- ciency and to the very low contribution to the fusion rate -100 from react..."


### Step 5: Modeling

Import needed libraries and read the training data created in the above steps

In [ ]:
import pandas as pd

# Replace 'example.csv' with the path to your actual file
file_path = 'training_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(data.head())

In [ ]:
sentences = data['content'].values
print(len(sentences))
print(sentences[0])

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3"
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("what is an atom?")

In [ ]:
ids = []

In [ ]:
texts = []
for i in range(0, len(sentences)):
    doc = sentences[i].split(" ")
    for j in range(0, len(doc), 300):
        temp = ' '.join(doc[j:j+300])
        texts.append(temp)

In [ ]:
for i in range(len(texts)):
    ids.append(str(i))

In [ ]:
import chromadb

# Create a Client Connection
# To load/persist db use db location as argument in Client method

client = chromadb.PersistentClient(path="./db")

# Create/Fetch a collection
collection = client.get_or_create_collection(name="test_vector_store")

In [ ]:
collection.add(documents = texts, ids=ids)

In [ ]:
question = "In the context of the palladium/hydrogen system, what are the observed effects of introducing unsaturated and saturated hydrocarbons into the hydrogen gas stream on the hydrogen transfer through a palladium membrane?"

In [ ]:
def getContext(q):
    results = collection.query(
    query_texts=[q],
    n_results=4
    )
    context = " ".join(results['documents'][0][0:4])
    print(context)
    return context

In [ ]:
print(results)

In [ ]:
#print(results['documents'][0][3])

context = " ".join(results['documents'][0][0:4])
print(context)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
chain = prompt | model | parser

ctx = getContext(question)

chain.invoke({"context": ctx, "question": question})

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(data['content'])

In [ ]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about Low Energy Nuclear Reactions",
)

In [ ]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")

In [ ]:
testset.save("test-set.jsonl")

In [ ]:
def answer_fn(question, history=None):
    ctx = getContext(question)
    return chain.invoke({"context": ctx, "question": question})

In [ ]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

In [ ]:
display(report)